# YouTube RAG Pipeline

Pipeline
1. **Loading Data**
2. **Chunking**
3. **Embeddings** (OpenAI-Embeddings)
4. **VectorDB** (Chroma)
5. **Retriever**
6. **LLM** (OpenAI Chat-Model)
7. **Chain** (Conversational Retrieval)
8. **Memory**


In [ ]:
!pip install -q --upgrade langchain langchain-openai langchain-community langchain-text-splitters chromadb tiktoken python-dotenv

In [ ]:
from pathlib import Path

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [ ]:
import os
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
    print("✅ API Key loaded successfully!")


✅ API Key loaded successfully!


## 1. Loading Data

Lege deine Transkript-Dateien als **`.txt`** in einen Ordner, z.B. `data/`.
Jede Datei kann z.B. der Transkripttext eines YouTube-Videos sein.

In [ ]:
# Ordner für Transkripte
DATA_DIR = Path("data")  # passe den Pfad an, falls nötig
DATA_DIR.mkdir(exist_ok=True)

def load_transcripts(data_dir: Path):
    docs = []
    for path in data_dir.glob('*.txt'):
        with path.open('r', encoding='utf-8') as f:
            text = f.read()
        if not text.strip():
            continue
        docs.append(
            Document(
                page_content=text,
                metadata={"source": path.name}
            )
        )
    return docs

documents = load_transcripts(DATA_DIR)
print(f"Geladene Dokumente: {len(documents)}")
if documents:
    print("Beispiel-Dokument:")
    print("Quelle:", documents[0].metadata)
    print(textwrap.shorten(documents[0].page_content, width=400, placeholder=" ..."))

## 2. Chunking

Wir zerlegen die langen Transkripte in kleinere Textstücke (Chunks), damit die Vektor-Suche besser funktioniert.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Größe eines Chunks (Tokens/Zeichen-Nähe)
    chunk_overlap=200,    # Überlappung, damit der Kontext nicht abreißt
    separators=["\n\n", "\n", ".", " ", ""]
)

splits = text_splitter.split_documents(documents)
print(f"Anzahl Chunks: {len(splits)}")
if splits:
    print("Beispiel-Chunk:")
    print("Quelle:", splits[0].metadata)
    print(textwrap.shorten(splits[0].page_content, width=300, placeholder=" ..."))

## 3. Embeddings & 4. VectorDB (Chroma)

Wir erzeugen Embeddings mit einem OpenAI-Embedding-Modell und speichern diese in einer Chroma-Datenbank.

In [ ]:
# Embedding-Modell
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Chroma-DB erstellen
PERSIST_DIR = "chroma_db"

vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=PERSIST_DIR
)

vectordb.persist()
print("✅ Vektor-Datenbank erstellt und gespeichert.")

## 5. Retriever

Der Retriever holt zu einer Frage die **relevantesten Chunks** aus der VectorDB.

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 4})
print("✅ Retriever bereit.")

## 6. LLM, 7. Chain & 8. Memory

Wir nutzen ein schnelles OpenAI-Chat-Modell (`gpt-4.1-mini`) und bauen eine
**ConversationalRetrievalChain**, die auch einen Chat-Verlauf (Memory) verwendet.

Der Chat-Verlauf wird in einer Python-Liste `chat_history` gespeichert.

In [ ]:
# LLM
llm = ChatOpenAI(
    model="gpt-4.1-mini",  # gutes Preis/Leistungs-Verhältnis
    temperature=0.2,        # eher sachlich
)

# Conversational Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
)

# Einfache Memory-Struktur: Liste aus (user, assistant)-Nachrichten
chat_history = []

def ask(question: str):
    """Stellt eine Frage an die RAG-Chain und aktualisiert den Chat-Verlauf."""
    global chat_history
    result = qa_chain({
        "question": question,
        "chat_history": chat_history,
    })

    answer = result["answer"]
    source_docs = result.get("source_documents", [])

    # Chat-Verlauf updaten
    chat_history.append((question, answer))

    # Antwort + Quellen hübsch ausgeben
    print("Frage:\n", question)
    print("\nAntwort:\n", textwrap.fill(answer, width=100))
    if source_docs:
        print("\nVerwendete Quellen:")
        for i, d in enumerate(source_docs, 1):
            short = textwrap.shorten(d.page_content, width=120, placeholder=" ...")
            print(f"[{i}] {d.metadata.get('source')} :: {short}")

    return answer


## 9. Test: Frage an dein RAG-System stellen

Jetzt kannst du dein System testen. Formuliere eine Frage, die im Inhalt deiner Transkripte beantwortet werden kann.

In [ ]:
# Beispiel-Frage (anpassen!)
example_question = "Worum geht es in dem ersten Video grob?"
ask(example_question)